# Лекция №11. Случайный лес (Random Forest).

Является методом bagging (bootstrap aggregation).  
Строится N деревьев (параметр ```n_estimators```) и устраивается голосование. Побеждает класс, за который проголосовало больше всего деревьев (в случае классификации) или значение всех голосов усредняется и выдается в качестве ответа (в случае регрессии). Кол-во деревьев логичнее выбирать как можно большее, а затем по результатам отбрасывать лишние (т.е. например, при значении параметра ```warm_start=True``` при наращивании леса смотреть, после какого кол-ва деревьев точность перестала значительно увеличиваться).

Деревья обучаются не на всей выборке (иначе они были бы одинаковыми), а на **подвыборке**, т.е. из всех строк данных отбирается (чаще всего случайным образом) некоторое заранее заданное их кол-во (параметр ```max_samples```, работает только при указании признака ```bootstrap=True```). По Брейману, это (1-1/е)* N = 0.632* N от всех данных, однако это значение можно варьировать на свое усмотрение.

Также для каждого конкретного дерева выбираются не все признаки (параметр ```max_features```). Это делается для того, чтобы добиться декорреляции результатов разных деревьев. Т.е. чтобы деревья давали некоторую вариативность результатов. По Брейману, sqrt(M), где M - число признаков. 

Минимальное число наблюдений в узле (параметр ```min_samples_leaf```): выбирается исходя из соображений, чтобы не наступало переобучение.

**Out-of-bag**: Т.к. каждое дерево из леса обучается на не всех строках данных, то оставшиеся строки данных для этого конкретного дерева можно условно считать тестовой выборкой и предварительно проверять на них качество обучения отдельных деревьев. Эти данные и назваются out-of-bag (т.е. не попавшие в обучение).

**Проверка значимости признаков в случайных лесах**:  
* Считать как ранее значимость в каждом из деревьев, в которых признак использовался в качестве обучающего и складывать  
* Выбирается признак и его значение перемешиваются внутри столбца случайным образом. Такая таблица данных попадает на обученный классификатор и анализируются полученные результаты: чем сильнее упала точность, тем значимее был признак. Если точность осталась такой же (или даже возросла), значит признак не влияет (или даже портит) классификацию/регрессию и классификатор необходимо переобучить уже без него. Далее все значения признака становятся в изначальном порядке и эксперимент проводится со следующим признаком.

In [14]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn import metrics

In [15]:
df = pd.read_csv('Credit.csv', sep=';', encoding='cp1251')
df.head()

,кредит,клаcс,з_плата,возраст,кр_карта
0,1,2,2,2,1
1,0,2,1,2,0
2,0,4,1,1,1
3,1,2,2,2,0
4,1,3,2,1,0


In [16]:
# Правильный ответ записываем в вектор y
y = df[u'кредит']
# Удаляем колонку с правильным ответом из таблицы и перемещаем в отдельный вектор ответов X
X = df.drop(u'кредит', axis=1)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=19,
                                                    # доля объёма тестового множества
                                                    test_size=0.33)

In [18]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=42,
                               # опции, относящиеся к отдельным деревьям такаие же, как в tree.DecisionTreeClassifier
                               # число деревьев в лесу
                               n_estimators=30,
                               # функция для impurity ('gini' или 'entropy')
                               criterion='gini',
                               max_depth=5,
                               # Вычислять out-of-bag ошибку
                               oob_score=True,
                               # использовать результаты предыдущего вызова и нарастить предыдущий лес 
                               warm_start=False,
                               # веса классов для балансировки обучения (см. пояснение в лекции №10)
                               class_weight=None
                              )

In [19]:
model.fit(X_train, y_train)

c:\users\bokoser_user\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


RandomForestClassifier(max_depth=5, n_estimators=30, oob_score=True,
                       random_state=42)

In [20]:
y_pred = model.predict(X_test)
print(metrics.classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.86      0.88      0.87        65
           1       0.80      0.79      0.80        42

    accuracy                           0.84       107
   macro avg       0.83      0.83      0.83       107
weighted avg       0.84      0.84      0.84       107



Видим, что по f1-скору случаный лес дал прирост в качестве с 0.80 до 0.86 по сравнению с одним решающим деревом.

In [21]:
print('Out-of-bag score: {0}'.format(model.oob_score_))

Out-of-bag score: 0.8935185185185185


In [22]:
pd.DataFrame({'feature': X.columns,
              'importance': model.feature_importances_}).sort_values('importance', ascending=False)

,feature,importance
1,з_плата,0.530931
2,возраст,0.286164
0,клаcс,0.162403
3,кр_карта,0.020502
